# Install `simex`

In [1]:
%%bash
#clone repo of `sipecam-metadata-extactor` in server
#git clone git@github.com:CONABIO/sipecam-metadata-extractor.git ~/
cd ../src
#install in editable mode
pip install -q -e .

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


# List of files to extract metadata

In [1]:
%%bash
dir_with_sipecam_data=/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM
list_of_files_and_subdirectories_to_extract_metadata --input_directory $dir_with_sipecam_data

## Check

In [2]:
%%bash
ls -lh /shared_volume/sipecam_files_to_extract_metadata_from_*

-rw-r--r-- 1 root root 3.2M Dec 16 18:03 /shared_volume/sipecam_files_to_extract_metadata_from_16-12-2021.txt


In [3]:
%%bash
ls -lh /shared_volume/sipecam_subdirectories_*

-rw-r--r-- 1 root root 1.8K Dec 16 18:03 /shared_volume/sipecam_subdirectories_16-12-2021.txt


In [4]:
%%bash
wc -l /shared_volume/sipecam_files_to_extract_metadata_from_*

29079 /shared_volume/sipecam_files_to_extract_metadata_from_16-12-2021.txt


In [5]:
%%bash
wc -l /shared_volume/sipecam_subdirectories_*

20 /shared_volume/sipecam_subdirectories_16-12-2021.txt


In [6]:
%%bash
#check every first subdirectory has a JPG, WAV or AVI file
SAVEIFS=$IFS;IFS=$(echo -en "\n\b")
for f in $(cat /shared_volume/sipecam_subdirectories_*.txt)
  do echo "$f"
     ls "$f"|grep "[WAV|wav|JPG|jpg|AVI|avi]$"|head -n 1
done
IFS=$SAVEIFS

/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico
20210728_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Audible
20210728_093000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_2/Ultrasonico
20210728_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_2/Audible
20210728_105000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_3/Nueva carpeta/Unidad de USB
20210728_141000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Ultrasonico
20210729_102000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible
20210729_180000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_4/Audible
20210729_133000.WAV
/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_1/Audio

# Extract serial number and coordinates

In [26]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/"
extract_serial_number_and_coordinates_of_file --input_dir "$dir_to_be_processed"

In [31]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico"
extract_serial_number_and_coordinates_of_file --input_dir "$dir_to_be_processed"

In [34]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible"
extract_serial_number_and_coordinates_of_file --input_dir "$dir_to_be_processed"

## Example of dir with different serial for images

In [22]:
%%bash
dir_to_be_processed="/LUSTRE/MADMEX/tasks/2021/3_extract_metadata_sipecam_files/notebooks/read_image_metadata"
extract_serial_number_and_coordinates_of_file --input_dir "$dir_to_be_processed"

## Check

### Dir with image files

In [27]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/"
file_for_logs=$(find "$dir_to_be_processed" -name 'logs_simex_extract_serial_numbers_and_coordinates.logs')
cat "$file_for_logs"

2022-01-26 10:49:49,133 [INFO]  extraction of serial number and coordinates
2022-01-26 10:49:49,134 [INFO]  serial number and coordinates in /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/100RECNX_simex_extract_serial_numbers_and_coordinates.json
2022-01-26 10:49:49,141 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2022-01-26 10:49:49,496 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2022-01-26 10:49:49,497 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0055.JPG
2022-01-26 10:49:49,753 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0055.JPG
2022

In [28]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/"
file_json=$(find "$dir_to_be_processed" -name '*.json')
python3 -mjson.tool "$file_json"

{
    "SerialNumber": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG": "HLPXGM09048890"
    },
    "Coordinates": {}
}


### Dir with audio files

In [32]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico"
file_for_logs=$(find "$dir_to_be_processed" -name 'logs_simex_extract_serial_numbers_and_coordinates.logs')
cat "$file_for_logs"

2022-01-26 10:52:50,223 [INFO]  extraction of serial number and coordinates
2022-01-26 10:52:50,224 [INFO]  serial number and coordinates in /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/Ultrasonico_simex_extract_serial_numbers_and_coordinates.json
2022-01-26 10:52:50,232 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2022-01-26 10:52:50,448 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2022-01-26 10:52:50,448 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210805_180000.WAV
2022-01-26 10:52:50,659 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/

In [33]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico"
file_json=$(find "$dir_to_be_processed" -name '*.json')
python3 -mjson.tool "$file_json"

{
    "SerialNumber": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV": "2423C2045F256D74"
    },
    "Coordinates": {}
}


### Dir with audio files

In [35]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible"
file_for_logs=$(find "$dir_to_be_processed" -name 'logs_simex_extract_serial_numbers_and_coordinates.logs')
cat "$file_for_logs"

2022-01-26 11:01:07,200 [INFO]  extraction of serial number and coordinates
2022-01-26 11:01:07,201 [INFO]  serial number and coordinates in /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/Audible_simex_extract_serial_numbers_and_coordinates.json
2022-01-26 11:01:07,210 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV
2022-01-26 11:01:07,543 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV
2022-01-26 11:01:07,544 [INFO]  extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210827_200000.WAV
2022-01-26 11:01:07,809 [INFO]  SUCCESSFUL extraction of serial number of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/2021082

In [36]:
%%bash
dir_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible"
file_json=$(find "$dir_to_be_processed" -name '*.json')
python3 -mjson.tool "$file_json"

{
    "SerialNumber": {
        "/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Galacia 1339/1339_2/Audio/Audible/20210806_044000.WAV": "248D9B045EC9E6B1"
    },
    "Coordinates": {}
}


# Extract metadata and ingest it

In [12]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [13]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

In [14]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
extract_metadata_and_ingest_it --input_file "$file_to_be_processed"

## Check

In [15]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:34,559 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0049.JPG
2021-12-15 11:35:34,561 [INFO]  Read metadata of JPG
2021-12-15 11:35:34,561 [INFO]  {}
2021-12-15 11:35:34,561 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:34,562 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/JPG


In [16]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:35,463 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Camaras/1338_1/100RECNX/RCNX0012.AVI
2021-12-15 11:35:35,465 [INFO]  Read metadata of AVI
2021-12-15 11:35:35,465 [INFO]  {}
2021-12-15 11:35:35,465 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:35,465 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/AVI


In [17]:
%%bash
file_to_be_processed="/LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV"
dirname_file_to_be_processed=$(dirname "$file_to_be_processed")
dir_for_logs=$(find "$dirname_file_to_be_processed" -name 'logs_simex_extract_metadata_and_ingest')
filename_to_be_processed=$(basename "$file_to_be_processed")
cat "$dir_for_logs/$filename_to_be_processed.logs"

2021-12-15 11:35:35,910 [INFO]  extraction of metadata and ingestion of /LUSTRE/sacmod/SIPECAM/Entregas_2021/octubre_2021/SIPECAM/Playon 1338/Audio/1338_1/Ultrasonico/20210806_044000.WAV
2021-12-15 11:35:35,912 [INFO]  Read metadata of WAV
2021-12-15 11:35:35,912 [INFO]  {}
2021-12-15 11:35:35,912 [INFO]  Getting node nomenclature and cumulus name from zendro
2021-12-15 11:35:35,912 [INFO]  Copying file to 3/1_3_1_28/2021-10-01/WAV


# Notes:

One option:

Serial number + coordinates of image -> node, cumulus, date of deployment (all this from Zendro models)

but for audio we can't retrieve coordinates of recording.

Another option:

Serial number + date of file (audio, image, video) -> node, cumulus and date of deployment from deployment model of Zendro

